# F1 Analytics
Data Pipeline project by Elina Yancheva and Vladimir Stoyanov

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np


from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [ ]:
circuits = pd.read_csv('data/circuits.csv')
constructor_results = pd.read_csv('data/constructor_results.csv')
constructor_standings = pd.read_csv('data/constructor_standings.csv')
constructors = pd.read_csv('data/constructors.csv')
driver_standings = pd.read_csv('data/driver_standings.csv')
drivers = pd.read_csv('data/drivers.csv')
lap_times = pd.read_csv('data/lap_times.csv')
pit_stops = pd.read_csv('data/pit_stops.csv')
qualifying = pd.read_csv('data/qualifying.csv')
races = pd.read_csv('data/races.csv')
results = pd.read_csv('data/results.csv')
seasons = pd.read_csv('data/seasons.csv')
sprint_results = pd.read_csv('data/sprint_results.csv')
status = pd.read_csv('data/status.csv')

tables = {
    'circuits': circuits,
    'constructor_results': constructor_results,
    'constructor_standings': constructor_standings,
    'constructors': constructors,
    'driver_standings': driver_standings,
    'drivers': drivers,
    'lap_times': lap_times,
    'pit_stops': pit_stops,
    'qualifying': qualifying,
    'races': races,
    'results': results,
    'seasons': seasons,
    'sprint_results': sprint_results,
    'status': status
}

# Data Cleaning

Key relationships in the F1 dataset:

- races is our central table, containing information about each race event (1,125 races)
- drivers (859 entries) and constructors (212 teams) are our main entities
- results (26,519 entries) connects races, drivers, and constructors with race outcomes
- lap_times (575,029 entries) and pit_stops (10,990 entries) provide detailed information about each race and driver
- qualifying (10,254 entries) and sprint_results (300 entries) cover pre-race events

In [ ]:
def analyze_dataset(df, name):
    print(f"\n{'='*50}")
    print(f"Analysis for {name} dataset")
    print(f"{'='*50}")
    
    print("\n Basic Information:")
    print(f"Number of rows: {df.shape[0]}")
    print(f"Number of columns: {df.shape[1]}")
    
    missing_values = df.isnull().sum()
    missing_percentages = (missing_values / len(df)) * 100
    if missing_values.sum() != 0:
        print("\n Missing Values Analysis:")
        for col, missing in missing_values.items():
            if missing > 0:
                print(f"{col}: {missing} missing values ({missing_percentages[col]:.2f}%)")
    
    duplicates = df.duplicated().sum()
    if duplicates > 0:
        print(f"\n Duplicate Rows: {duplicates}")
        df = df.drop_duplicates()

datasets = {
    'circuits': circuits,
    'constructor_results': constructor_results,
    'constructor_standings': constructor_standings,
    'constructors': constructors,
    'driver_standings': driver_standings,
    'drivers': drivers,
    'lap_times': lap_times,
    'pit_stops': pit_stops,
    'qualifying': qualifying,
    'races': races,
    'seasons': seasons,
    'sprint_results': sprint_results,
    'status': status
}

for name, df in datasets.items():
    analyze_dataset(df, name)

## Invalid entries search

In [ ]:
def validate_f1_data(lap_times_df, results_df, qualifying_df, races_df, pit_stops_df):
    print("F1 Data Validation Report")
    print("=" * 30)
    
    issues = []
    print("\nQualifying Validation:")
    
    # Check for drivers with multiple qualifying times in same session
    qual_dupes = qualifying_df.groupby(['raceId', 'driverId']).size().reset_index(name='count')
    qual_dupes = qual_dupes[qual_dupes['count'] > 1]
    if not qual_dupes.empty:
        issues.append(f"Found {len(qual_dupes)} duplicate qualifying entries")
    else:
        print("No duplicate qualifying entries found!")
    
    print("\nRace Schedule Validation:")
    
    # Check for races scheduled on same date
    race_dates = races_df.groupby('date').size().reset_index(name='count')
    date_clashes = race_dates[race_dates['count'] > 1]
    if not date_clashes.empty:
        issues.append(f"Found {len(date_clashes)} date clashes in race schedule")
    else:
        print("No date clashes found!")
    
    print("\nValidation Summary:")
    if issues:
        print(f"\nFound {len(issues)} types of data issues:")
        for issue in issues:
            print(f"- {issue}")
    else:
        print("\nNo major data issues found!")
    
    return issues

issues = validate_f1_data(lap_times, results, qualifying, races, pit_stops)

In [ ]:
def validate_lap_times(lap_times_df, races_df, drivers_df):
    print("\nLap Times Validation:")
    print("-" * 30)
    
    # Check for zero or negative lap times
    zero_laps = lap_times_df[lap_times_df['milliseconds'] <= 0]
    if not zero_laps.empty:
        issues.append(f"Found {len(zero_laps)} zero or negative lap times")
    else: 
        print("No zero or negative lap times found")

    lap_times_df['minutes'] = lap_times_df['milliseconds'] / (1000 * 60)
    
    slow_laps = lap_times_df[lap_times_df['minutes'] > 5].copy()
    
    if not slow_laps.empty:
        # Group slow laps by race and nearby laps (within ±2 laps)
        # When a red flag or safety car occurs, drivers might be on different laps
        race_incidents = []
        for race_id in slow_laps['raceId'].unique():
            race_slow_laps = slow_laps[slow_laps['raceId'] == race_id]
            
            for _, incident_lap in race_slow_laps.iterrows():
                nearby_laps = race_slow_laps[
                    (race_slow_laps['lap'] >= incident_lap['lap'] - 2) & 
                    (race_slow_laps['lap'] <= incident_lap['lap'] + 2)
                ]
                
                if len(nearby_laps) > 5:  # If many drivers affected in nearby laps
                    race_incidents.append({
                        'raceId': race_id,
                        'lap_range': f"{nearby_laps['lap'].min()}-{nearby_laps['lap'].max()}",
                        'affected_drivers': len(nearby_laps),
                        'avg_time': nearby_laps['minutes'].mean(),
                        'laps': sorted(nearby_laps['lap'].unique())
                    })
        
        # Remove duplicates (same incident might be counted multiple times)
        unique_incidents = []
        processed_races = set()
        
        for incident in race_incidents:
            if incident['raceId'] not in processed_races:
                unique_incidents.append(incident)
                processed_races.add(incident['raceId'])
        
        # Identify legitimate vs suspicious slow laps
        legitimate_races = {incident['raceId'] for incident in unique_incidents}
        legitimate_slow = slow_laps[slow_laps['raceId'].isin(legitimate_races)]
        suspicious_slow = slow_laps[~slow_laps['raceId'].isin(legitimate_races)]
        
        legitimate_slow = legitimate_slow.merge(
            races_df[['raceId', 'name', 'year']], 
            on='raceId'
        ).merge(
            drivers_df[['driverId', 'forename', 'surname']], 
            on='driverId'
        )
        
        suspicious_slow = suspicious_slow.merge(
            races_df[['raceId', 'name', 'year']], 
            on='raceId'
        ).merge(
            drivers_df[['driverId', 'forename', 'surname']], 
            on='driverId'
        )
        
        print(f"\nFound {len(slow_laps)} lap times longer than 5 minutes")
        print(f"- {len(legitimate_slow)} likely due to red flags/safety car")
        print(f"- {len(suspicious_slow)} potentially suspicious\n")
        
        if unique_incidents:
            print("Probable red flag/safety car incidents:")
            for incident in unique_incidents:
                race_info = races_df[races_df['raceId'] == incident['raceId']].iloc[0]
                print(f"\nRace: {race_info['name']} {race_info['year']}")
                print(f"Lap range: {incident['lap_range']}")
                print(f"Affected drivers: {incident['affected_drivers']}")
                print(f"Average lap time: {incident['avg_time']:.2f} minutes")
                print(f"Affected laps: {incident['laps']}")
        
        if not suspicious_slow.empty:
            print("\nSuspicious individual slow laps:")
            for _, lap in suspicious_slow.iterrows():
                print(f"Race: {lap['name']} {lap['year']}, "
                      f"Driver: {lap['forename']} {lap['surname']}, "
                      f"Lap: {lap['lap']}, "
                      f"Time: {lap['minutes']:.2f} minutes")
    
    return legitimate_slow, suspicious_slow

legitimate_slow_laps, suspicious_slow_laps = validate_lap_times(lap_times, races, drivers)

Lap Times Validation:
------------------------------

Found 692 lap times longer than 5 minutes
- 608 likely due to red flags/safety car
- 84 potentially suspicious

Based on the analysis of Formula 1 lap times, there were 692 laps identified as longer than 5 minutes, with 608 of these explained by red flags or safety car periods (which force drivers to slow down significantly), as evidenced by multiple drivers having similarly long lap times during those specific race laps. The remaining 84 "suspicious" individual slow laps, while initially flagged as potential anomalies, appear to be legitimate race incidents (like pit stops, mechanical issues, or on-track incidents) rather than data errors, which is supported by Google fact-checking of several cases. 

# Data Transformation

## Dataset overview

In [ ]:
for name, df in tables.items():
    print(f"\n{name} dataset:")
    display(df.tail())

In [ ]:
def aggregate_performance_metrics():
    """
    Creates summary statistics and performance metrics.
    """
    
    driver_season_stats = results.merge(races[['raceId', 'year']], on='raceId')
    invalid_position = driver_season_stats[driver_season_stats['position'] == '\\N']
    print("\nDNFs with reasons:")
    display(invalid_position.merge(status, on='statusId')[['raceId', 'year', 'driverId', 'position', 'status']])
    
    # count DNFs(Did Not Finish)
    dnf_count = invalid_position.groupby(['year', 'driverId']).agg({'position': 'count'}).reset_index().rename(columns={'position': 'dnf_count'})
    
    # Filter valid entries and convert datatypes
    driver_season_stats = driver_season_stats[
        (driver_season_stats['position'] != '\\N') & 
        (driver_season_stats['milliseconds'] != '\\N')
    ]
    driver_season_stats['position'] = driver_season_stats['position'].astype(int)
    driver_season_stats['points'] = driver_season_stats['points'].astype(float)
    driver_season_stats['milliseconds'] = driver_season_stats['milliseconds'].astype(float)
    
    # Driver performance by season
    driver_season_summary = driver_season_stats.groupby(['year', 'driverId']).agg({
        'points': 'sum',
        'position': ['mean', 'min'],
        'milliseconds': ['mean', 'std']
    }).reset_index()
    
    driver_season_summary.columns = ['year', 'driverId', 'total_points', 
                                   'avg_position', 'best_position',
                                   'avg_race_time', 'race_time_std']
    
    # Now merge with DNF counts
    driver_season_summary = driver_season_summary.merge(dnf_count, on=['year', 'driverId'], how='left').fillna(0)
    driver_season_summary = driver_season_summary.merge(drivers[['driverId', 'forename', 'surname']], on='driverId').drop('driverId', axis=1)
    
    # Constructor performance trends
    constructor_trends = results.merge(races[['raceId', 'year']], on='raceId')
    constructor_summary = constructor_trends.groupby(['year', 'constructorId']).agg({
        'points': 'sum',
        'laps': 'sum'
    }).reset_index()
    constructor_summary = constructor_summary.merge(constructors[['constructorId', 'name']], on='constructorId').drop('constructorId', axis=1)
    
    return {
        'driver_season_stats': driver_season_summary,
        'constructor_trends': constructor_summary
    }

for k, a in aggregate_performance_metrics().items():
    print(f"\n{k}:")
    display(a)

In [ ]:
race_results = results.merge(races[['raceId', 'year']], on='raceId')
race_results = race_results.merge(constructors[['constructorId', 'name']], on='constructorId')

# average race time per year across all constructors
race_times = race_results[race_results['milliseconds'] != '\\N'].copy()
race_times['hours'] = race_times['milliseconds'].astype(float) / (1000 * 60 * 60)  

yearly_times = race_times.groupby('year')['hours'].agg(['mean', 'std', 'count']).reset_index()
yearly_times = yearly_times[yearly_times['count'] > 5]  # Filter years with enough data

plt.figure(figsize=(15, 8))
plt.errorbar(yearly_times['year'], 
            yearly_times['mean'], 
            yerr=yearly_times['std'],
            alpha=0.5,
            capsize=3,
            label='Standard deviation')

plt.plot(yearly_times['year'], yearly_times['mean'], linewidth=2, label='Average race time')

plt.title('Average F1 Race Duration Over the Years')
plt.xlabel('Year')
plt.ylabel('Race Duration (hours)')
plt.grid(True, alpha=0.3)
plt.legend()

# trend comparison annotations
earliest_time = yearly_times.iloc[0]['mean']
latest_time = yearly_times.iloc[-1]['mean']
time_diff = latest_time - earliest_time

print("\nRace Duration Changes:")
print(f"Average race time in {yearly_times.iloc[0]['year']}: {earliest_time:.2f} hours")
print(f"Average race time in {yearly_times.iloc[-1]['year']}: {latest_time:.2f} hours")
print(f"Change: {time_diff:.2f} hours ({(time_diff/earliest_time * 100):.1f}% change)")

# Show fastest and slowest years
fastest_year = yearly_times.loc[yearly_times['mean'].idxmin()]
slowest_year = yearly_times.loc[yearly_times['mean'].idxmax()]
print(f"\nFastest average races: {fastest_year['year']} ({fastest_year['mean']:.2f} hours)")
print(f"Slowest average races: {slowest_year['year']} ({slowest_year['mean']:.2f} hours)")

plt.show()

## Analysis of Race Duration and Calendar Evolution:
F1 races have become significantly shorter and more standardized since the 1950s, with average race duration dropping from 2.66 hours to 1.58 hours (40.5% reduction). The most extreme year was 1954 with 3.53-hour races, while 1991 saw the shortest average at 1.41 hours. This trend toward shorter races aligns with modern F1's focus on sprint formats and tighter racing. Meanwhile, the calendar has expanded dramatically, from just 7-8 races in the early years to over 20 races today, showing F1's growth into a truly global championship with a much more demanding schedule

In [ ]:
# Count races per season
races_per_year = races.groupby('year').size().reset_index(name='race_count').astype(int)

plt.figure(figsize=(15, 8))
plt.plot(races_per_year['year'], races_per_year['race_count'], 
        marker='o', linewidth=2, markersize=4)
plt.grid(True, alpha=0.3)
plt.title('Number of Races per F1 Season')
plt.xlabel('Year')
plt.ylabel('Number of Races')
plt.show()

print(f"First season (1950): {races_per_year.iloc[0]['race_count']} races")
print(f"Latest season (2024): {races_per_year.iloc[-1]['race_count']} races")
print(f"Most races: {races_per_year['race_count'].max()} (Year: {races_per_year.loc[races_per_year['race_count'].idxmax(), 'year']})")
print(f"Fewest races: {races_per_year['race_count'].min()} (Year: {races_per_year.loc[races_per_year['race_count'].idxmin(), 'year']})")

The races may have been shorthened but luckily their count has been increasing over the years.

## Drivers comparison

In [ ]:
def get_driver_stats_comparison(drivers_df, results_df, qualifying_df, races_df):
    schumacher_id = drivers_df[
        (drivers_df['forename'] == 'Michael') & 
        (drivers_df['surname'] == 'Schumacher')
    ]['driverId'].iloc[0]
    
    hamilton_id = drivers_df[
        (drivers_df['forename'] == 'Lewis') & 
        (drivers_df['surname'] == 'Hamilton')
    ]['driverId'].iloc[0]
    
    def get_driver_stats(driver_id):
        wins = results_df[
            (results_df['driverId'] == driver_id) & 
            (results_df['position'] == '1')
        ].shape[0]
        
        races = results_df[results_df['driverId'] == driver_id].shape[0]
        
        qualifying_sessions = qualifying_df[qualifying_df['driverId'] == driver_id].shape[0]
        
        poles = qualifying_df[
            (qualifying_df['driverId'] == driver_id) & 
            (qualifying_df['position'] == 1)
        ].shape[0]
        
        points = results_df[results_df['driverId'] == driver_id]['points'].sum()
        
        podiums = results_df[
            (results_df['driverId'] == driver_id) & 
            (results_df['position'].isin(['1', '2', '3']))
        ].shape[0]
        
        dnfs = results_df[
            (results_df['driverId'] == driver_id) & 
            (results_df['positionText'] == 'R')
        ].shape[0]
        
        return {
            'wins': wins,
            'races': races,
            'qualifying': qualifying_sessions,
            'poles': poles,
            'podiums': podiums,
            'dnf': dnfs,
        }
    
    schumacher_stats = get_driver_stats(schumacher_id)
    hamilton_stats = get_driver_stats(hamilton_id)
    
    # Print comparison
    print("Statistical Comparison: Schumacher vs Hamilton")
    print("=" * 50)
    for metric in schumacher_stats.keys():
        print(f"{metric.upper():<20} {schumacher_stats[metric]:<15} {hamilton_stats[metric]}")
    
    return schumacher_stats, hamilton_stats

schumacher_stats, hamilton_stats = get_driver_stats_comparison(
    drivers, results, qualifying, races
)

In [ ]:
def create_f1_comparison_plots(drivers_df, results_df, qualifying_df, races_df):
    schumacher_id = drivers_df[
        (drivers_df['forename'] == 'Michael') & 
        (drivers_df['surname'] == 'Schumacher')
    ]['driverId'].iloc[0]
    
    hamilton_id = drivers_df[
        (drivers_df['forename'] == 'Lewis') & 
        (drivers_df['surname'] == 'Hamilton')
    ]['driverId'].iloc[0]
    
    verstappen_id = drivers_df[
        (drivers_df['forename'] == 'Max') & 
        (drivers_df['surname'] == 'Verstappen')
    ]['driverId'].iloc[0]
    
    plt.style.use('dark_background')
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 14), height_ratios=[3, 1])
    
    # First plot (Career Statistics)
    def get_driver_stats(driver_id):
        driver_years = results_df[results_df['driverId'] == driver_id].merge(
            races_df[['raceId', 'year']], on='raceId'
        )['year'].nunique()
        
        wins = results_df[
            (results_df['driverId'] == driver_id) & 
            (results_df['position'] == '1')
        ].shape[0]
        
        races = results_df[results_df['driverId'] == driver_id].shape[0]
        qualifying_sessions = qualifying_df[qualifying_df['driverId'] == driver_id].shape[0]
        
        poles = qualifying_df[
            (qualifying_df['driverId'] == driver_id) & 
            (qualifying_df['position'] == 1)
        ].shape[0]
        
        podiums = results_df[
            (results_df['driverId'] == driver_id) & 
            (results_df['position'].isin(['1', '2', '3']))
        ].shape[0]
        
        return {
            'YEARS RACING': driver_years,
            'WINS': wins,
            'RACES': races,
            'QUALIFYING': qualifying_sessions,
            'POLES': poles,
            'PODIUMS': podiums,
        }
    
    # Get career stats
    schumacher_stats = get_driver_stats(schumacher_id)
    hamilton_stats = get_driver_stats(hamilton_id)
    verstappen_stats = get_driver_stats(verstappen_id)
    
    metrics = list(schumacher_stats.keys())
    schumacher_values = list(schumacher_stats.values())
    hamilton_values = list(hamilton_stats.values())
    verstappen_values = list(verstappen_stats.values())
    
    # Plot career stats
    y_pos = np.arange(len(metrics))
    bar_height = 0.25
    
    ax1.barh(y_pos - bar_height, schumacher_values, bar_height, 
             label='Schumacher', color='#ff0000')
    ax1.barh(y_pos, hamilton_values, bar_height, 
             label='Hamilton', color='#2dd4bf')
    ax1.barh(y_pos + bar_height, verstappen_values, bar_height, 
             label='Verstappen', color='blue')
    
    def add_value_labels(values, position, ax):
        for i, v in enumerate(values):
            ax.text(v, i + position, str(v), 
                   ha='left', va='center', color='white', fontsize=9)
    
    add_value_labels(schumacher_values, -bar_height, ax1)
    add_value_labels(hamilton_values, 0, ax1)
    add_value_labels(verstappen_values, bar_height, ax1)
    
    ax1.set_yticks(y_pos)
    ax1.set_yticklabels(metrics, fontsize=10)
    ax1.invert_yaxis()
    
    # Second plot (Points Comparison)
    def get_points_by_season(driver_id):
        points_data = results_df[results_df['driverId'] == driver_id].merge(
            races_df[['raceId', 'year']], on='raceId'
        )
        return points_data.groupby('year')['points'].sum().reset_index()
    
    schumacher_points = get_points_by_season(schumacher_id)
    hamilton_points = get_points_by_season(hamilton_id)
    verstappen_points = get_points_by_season(verstappen_id)
    
    ax2.bar(1, schumacher_points['points'].sum(), width=0.8, 
            color='#ff0000', label='Schumacher')
    ax2.bar(2, hamilton_points['points'].sum(), width=0.8, 
            color='#2dd4bf', label='Hamilton')
    ax2.bar(3, verstappen_points['points'].sum(), width=0.8, 
            color='blue', label='Verstappen')
    
    for i, points in enumerate([schumacher_points['points'].sum(), 
                              hamilton_points['points'].sum(), 
                              verstappen_points['points'].sum()], 1):
        ax2.text(i, points, f'{points:,.0f}', 
                ha='center', va='bottom', color='white', fontsize=10)
    
    ax2.set_xticks([1, 2, 3])
    ax2.set_xticklabels(['Schumacher', 'Hamilton', 'Verstappen'])
    ax2.set_title('Total Career Points', pad=20, color='white', fontsize=12)
    
    for ax in [ax1, ax2]:
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        ax.spines['bottom'].set_visible(False)
        ax.spines['left'].set_visible(False)
        ax.xaxis.set_visible(False)
        ax.set_facecolor('#1f2937')
    
    fig.patch.set_facecolor('#1f2937')
    
    ax1.set_title('Career Statistics Comparison', 
                  pad=20, color='white', fontsize=14)
    
    ax1.legend(loc='upper right', frameon=False, 
              fontsize=10, bbox_to_anchor=(1, 1.1))
    
    plt.tight_layout()
    return fig

fig = create_f1_comparison_plots(drivers, results, qualifying, races)
plt.show()

## Alonso over the years

In [ ]:
def create_alonso_seasons_plot(drivers_df, results_df, races_df, constructors_df):
    alonso_id = drivers_df[
        (drivers_df['forename'] == 'Fernando') & 
        (drivers_df['surname'] == 'Alonso')
    ]['driverId'].iloc[0]
    
    alonso_results = results_df[results_df['driverId'] == alonso_id].merge(
        races_df[['raceId', 'year']], on='raceId'
    ).merge(
        constructors_df[['constructorId', 'name']], on='constructorId'
    )
    
    # Group by year and team
    season_data = alonso_results.groupby(['year', 'name'])['points'].sum().reset_index()
    
    season_teams = season_data.groupby('year')['name'].agg(list).reset_index()
    season_points = season_data.groupby('year')['points'].sum().reset_index()
    
    season_summary = season_points.merge(season_teams, on='year')
    
    team_colors = {
        'Minardi': '#000000',
        'Renault': '#FFD700',
        'McLaren': '#FF8C00',
        'Ferrari': '#DC143C',
        'Alpine F1 Team': '#0090FF',
        'Aston Martin': '#006F62'
    }
    
    plt.style.use('dark_background')
    fig, ax = plt.subplots(figsize=(15, 8))
    
    bars = ax.bar(season_summary['year'], season_summary['points'])
    
    for bar, teams in zip(bars, season_summary['name']):
        if len(teams) == 1:
            bar.set_color(team_colors.get(teams[0], '#666666'))
        else:
            bar.set_color('#666666')  # Multiple teams in one season
    
    ax.set_facecolor('#1f2937')
    fig.patch.set_facecolor('#1f2937')
    
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['left'].set_visible(False)
    
    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height,
                f'{int(height)}',
                ha='center', va='bottom', color='white')
    
    plt.xticks(season_summary['year'], rotation=45)
    
    plt.title("Fernando Alonso: Points per Season", 
              pad=20, color='white', fontsize=14)
    
    legend_elements = [plt.Rectangle((0,0),1,1, facecolor=color, label=team)
                      for team, color in team_colors.items()]
    ax.legend(handles=legend_elements, loc='upper right', 
             bbox_to_anchor=(1, 1.15), ncol=3, frameon=False)
    
    ax.grid(axis='y', linestyle='--', alpha=0.2)
    
    plt.tight_layout()
    return fig

fig = create_alonso_seasons_plot(drivers, results, races, constructors)
plt.show()

## Load datasets from pipeline output

In [ ]:
championship_winners = pd.read_parquet('data/championship_winners.parquet')

print("\nF1 Championship Winners Dataset:")
print("=" * 50)
print(championship_winners.head())

print("\nTotal number of championships:", len(championship_winners))
print("\nNumber of championships by driver:")
print(championship_winners['driver_name'].value_counts().head())
print("\nNumber of championships by nationality:")
print(championship_winners['nationality'].value_counts().head())

In [ ]:
constructor_champions = pd.read_parquet('data/constructor_champions.parquet')

print("\nF1 Constructor Championship Winners Dataset:")
print("=" * 50)
print(constructor_champions.head())

print("\nTotal number of championships:", len(constructor_champions))
print("\nNumber of championships by constructor:")
print(constructor_champions['name'].value_counts().head())
print("\nNumber of championships by nationality:")
print(constructor_champions['nationality'].value_counts().head())

most_recent = constructor_champions.iloc[-1]
print(f"\nMost recent constructor champion ({most_recent['year']}): {most_recent['name']}")

In [ ]:
def plot_championships_by_nationality():
    plt.style.use('dark_background')
    
    constructor_counts = constructor_champions['nationality'].value_counts()
    
    fig1, ax1 = plt.subplots(figsize=(12, 8))
    patches1, texts1, autotexts1 = ax1.pie(
        constructor_counts,
        labels=constructor_counts.index,
        autopct=lambda pct: f'{pct:.1f}%\n({int(pct/100.*sum(constructor_counts))})',
        colors=['#FF0000' if nat == 'Italian' else f'#{hash(nat) % 0xFFFFFF:06x}' for nat in constructor_counts.index],
        startangle=90,
        labeldistance=1.1,  # Move labels further out
        pctdistance=0.75,   # Move percentages closer to edge
        explode=[0.05] * len(constructor_counts)  # Separate slices
    )
    plt.setp(autotexts1, size=9, weight="bold", color='white')
    plt.setp(texts1, size=10, color='white')
    ax1.set_title('Constructor Championships by Nationality', pad=20, size=14)
    fig1.patch.set_facecolor('#1f2937')
    ax1.set_facecolor('#1f2937')
    plt.show()

plot_championships_by_nationality()

## Position gains

In [ ]:
def find_best_position_gains(qualifying_df, results_df, drivers_df, races_df):
    position_changes = qualifying_df.merge(
        results_df[['raceId', 'driverId', 'positionOrder']], 
        on=['raceId', 'driverId']
    )
    
    position_changes = position_changes.merge(
        drivers_df[['driverId', 'forename', 'surname']], 
        on='driverId'
    )
    
    position_changes = position_changes.merge(
        races_df[['raceId', 'year', 'name']], 
        on='raceId'
    )
    
    # Convert qualifying position to numeric
    position_changes['position_q'] = pd.to_numeric(position_changes['position'])
    
    # Calculate position gain (qualifying position - finish position)
    position_changes['positions_gained'] = position_changes['position_q'] - position_changes['positionOrder']
    
    gains = position_changes[[
        'year', 'name', 'forename', 'surname', 
        'position_q', 'positionOrder', 'positions_gained'
    ]].copy()
    gains['driver_name'] = gains['forename'] + ' ' + gains['surname']
    
    # Filter for only positive gains (moving up the order)
    gains = gains[gains['positions_gained'] > 0]
    
    gains = gains.sort_values('positions_gained', ascending=False)
    top_10_gains = gains.head(10)
    
    result = top_10_gains[[
        'driver_name', 'year', 'name', 
        'position_q', 'positionOrder', 'positions_gained'
    ]].rename(columns={
        'name': 'race',
        'position_q': 'qualifying_position',
        'positionOrder': 'finishing_position'
    })
    
    return result

top_gains = find_best_position_gains(qualifying, results, drivers, races)

print("\nTop 10 Greatest Comebacks in F1 History (Qualifying to Race Finish)")
print("=" * 80)
display(top_gains.sort_values(by=['positions_gained', 'year'], ascending=False) .head(n=10))

In [ ]:
def plot_top_position_gains(qualifying_df, results_df, drivers_df, races_df):
    # Get only top 5 from our previous function
    top_gains = find_best_position_gains(qualifying_df, results_df, drivers_df, races_df).head(5)
    
    plt.style.use('dark_background')
    fig, ax = plt.subplots(figsize=(10, 6), dpi=100)
    
    start_color = '#ff6b6b'  # Red for qualifying
    end_color = '#4ecdc4'    # Green for finish
    line_color = '#666666'   # Grey for connecting lines
    
    x_positions = [0, 1]  # 0 for qualifying, 1 for race finish
    
    for idx, row in top_gains.iterrows():
        # Plot lines between qualifying and race positions
        plt.plot(x_positions, 
                [row['qualifying_position'], row['finishing_position']], 
                color=line_color, 
                label=f"{row['driver_name'].split(' ')[-1]} ({row['year']})")
        
        plt.scatter(0, row['qualifying_position'], color=start_color, s=100, zorder=5)
        plt.scatter(1, row['finishing_position'], color=end_color, s=100, zorder=5)
        
        plt.text(1.1, row['finishing_position'], 
                f"+{int(row['positions_gained'])}", 
                color=end_color, 
                va='center')

    ax.set_xlim(-0.2, 1.5)
    ax.set_xticks(x_positions)
    ax.set_xticklabels(['Qualifying', 'Race Finish'])
    
    # Set y-axis (reversed, as P1 should be at top)
    max_position = max(top_gains['qualifying_position'].max(), top_gains['finishing_position'].max())
    ax.set_ylim(max_position + 1, 0)
    ax.set_yticks(range(1, max_position + 1))
    ax.set_yticklabels([f'P{pos}' for pos in range(1, max_position + 1)])
    
    plt.title('Top 5 Greatest F1 Position Gains', pad=20, fontsize=12)
    plt.legend(loc='center right', bbox_to_anchor=(1.3, 0.5), frameon=False)
    
    ax.set_facecolor('#1f2937')
    fig.patch.set_facecolor('#1f2937')
    ax.grid(True, axis='y', linestyle='--', alpha=0.2)
    
    for spine in ax.spines.values():
        spine.set_visible(False)
    
    plt.tight_layout()
    return fig

fig = plot_top_position_gains(qualifying, results, drivers, races)
plt.show()

# Race prediction model 
Random forest model trained to predict finish position based on several factors - e.g. grid position, qualifying time, fastest lap time, etc. 

In [ ]:
def prepare_race_prediction_data(qualifying_df, results_df, drivers_df, constructors_df, races_df):
    race_data = results_df.merge(
        races_df[['raceId', 'circuitId', 'year']], 
        on='raceId'
    )
    
    race_data = race_data.merge(
        qualifying_df[['raceId', 'driverId', 'position']], 
        on=['raceId', 'driverId'],
        suffixes=('_finish', '_quali')
    )
    
    race_data = race_data.merge(drivers_df[['driverId', 'nationality']], on='driverId')
    race_data = race_data.merge(constructors_df[['constructorId', 'nationality']], 
                               on='constructorId', suffixes=('_driver', '_constructor'))
    
    def create_historical_features(df):
        df = df.sort_values('year')
        
        # Last 3 races performance
        df['last_3_avg_position'] = df.groupby('driverId')['positionOrder'].transform(
            lambda x: x.shift().rolling(3).mean()
        )
        
        # Constructor's last 3 races
        df['constructor_last_3_avg'] = df.groupby('constructorId')['positionOrder'].transform(
            lambda x: x.shift().rolling(3).mean()
        )
        
        df['track_history_avg'] = df.groupby(['driverId', 'circuitId'])['positionOrder'].transform(
            lambda x: x.shift().mean()
        )
        
        # Qualifying performance vs race performance ratio
        df['quali_race_ratio'] = df['position_quali'] / df['positionOrder']
        
        # Driver's performance at this circuit
        df['circuit_win_rate'] = df.groupby(['driverId', 'circuitId'])['positionOrder'].transform(
            lambda x: (x.shift() == 1).mean()
        )
        
        # Constructor's performance at this circuit
        df['constructor_circuit_avg'] = df.groupby(['constructorId', 'circuitId'])['positionOrder'].transform(
            lambda x: x.shift().mean()
        )
        
        return df
    
    features_df = race_data.pipe(create_historical_features)
    
    X = features_df[[
        'grid',
        'position_quali',
        'last_3_avg_position',
        'constructor_last_3_avg',
        'track_history_avg',
        'quali_race_ratio',
        'circuit_win_rate',
        'constructor_circuit_avg'
    ]].fillna(-1)
    
    y = features_df['positionOrder']
    
    return X, y

def create_race_prediction_model():
    X, y = prepare_race_prediction_data(qualifying, results, drivers, constructors, races)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    
    cv_scores = cross_val_score(model, X, y, cv=5, scoring='neg_mean_squared_error')
    cv_rmse = np.sqrt(-cv_scores)
    
    print("Test Set Results:")
    print(f"Mean Squared Error: {mse:.2f}")
    print(f"Mean Absolute Error: {mae:.2f}")
    print(f"\nCross-validation RMSE: {cv_rmse.mean():.2f} (+/- {cv_rmse.std() * 2:.2f})")
    
    feature_importance = pd.DataFrame({
        'feature': X.columns,
        'importance': model.feature_importances_
    }).sort_values('importance', ascending=False)
    
    print("\nFeature Importance:")
    print(feature_importance)
    
    return model, feature_importance

model, feature_importance = create_race_prediction_model()

In [ ]:
def plot_race_prediction_analysis(model, feature_importance):
    X, y = prepare_race_prediction_data(qualifying, results, drivers, constructors, races)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    y_pred = model.predict(X_test)
    
    plt.style.use('dark_background')
    fig = plt.figure(figsize=(20, 15))
    
    ax1 = plt.subplot(2, 2, 1)
    feature_importance_plot = feature_importance.copy()
    feature_importance_plot['feature'] = feature_importance_plot['feature'].map({
        'grid': 'Grid Position',
        'position_quali': 'Qualifying Position',
        'last_3_avg_position': 'Last 3 Races Avg',
        'constructor_last_3_avg': 'Constructor Last 3 Races',
        'track_history_avg': 'Track History',
        'quali_race_ratio': 'Quali/Race Ratio',
        'circuit_win_rate': 'Circuit Win Rate',
        'constructor_circuit_avg': 'Constructor at Circuit'
    })
    
    ax1.barh(y=range(len(feature_importance_plot)), 
             width=feature_importance_plot['importance'],
             color='#2dd4bf')
    ax1.set_yticks(range(len(feature_importance_plot)))
    ax1.set_yticklabels(feature_importance_plot['feature'])
    ax1.set_title('Feature Importance in Race Position Prediction', pad=20)
    ax1.set_xlabel('Importance Score')
    
    # predicted vs actual plot
    ax2 = plt.subplot(2, 2, 2)
    scatter = ax2.scatter(y_test, y_pred, alpha=0.5, color='#2dd4bf')
    ax2.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 
             'r--', lw=2, label='Perfect Prediction')
    ax2.set_xlabel('Actual Finish Position')
    ax2.set_ylabel('Predicted Finish Position')
    ax2.set_title('Predicted vs Actual Finish Positions', pad=20)
    ax2.legend()
    
    
    for ax in [ax1, ax2]:
        ax.set_facecolor('#1f2937')
        for spine in ax.spines.values():
            spine.set_color('#666666')
        ax.grid(True, alpha=0.2)
        
    fig.patch.set_facecolor('#1f2937')
    plt.tight_layout()
    
    # position-specific accuracy
    print("\nAccuracy by Position Range:")
    position_ranges = [(1,3), (4,10), (11,20)]
    for start, end in position_ranges:
        mask = (y_test >= start) & (y_test <= end)
        if mask.any():
            range_mae = mean_absolute_error(y_test[mask], y_pred[mask])
            print(f"\nPositions {start}-{end}:")
            print(f"Mean Absolute Error: {range_mae:.2f} positions")
            print(f"Samples: {mask.sum()}")
    
    return fig

fig = plot_race_prediction_analysis(model, feature_importance)
plt.show()

In [ ]:
def predict_race_finish(model, driver_scenario):
    X_predict = pd.DataFrame([driver_scenario])
    
    predicted_position = model.predict(X_predict)[0]
    
    print("\nRace Scenario Analysis:")
    print("-" * 30)
    for feature, value in driver_scenario.items():
        print(f"{feature}: {value}")
    
    print(f"\nPredicted Finish Position: {predicted_position:.1f}")
    
    return predicted_position

# Let's simulate a top driver starting from P6
scenario = {
    'grid': 6,  # Starting from P6
    'position_quali': 6,  # Qualified P6
    'last_3_avg_position': 3.0,  # Average P3 in last 3 races
    'constructor_last_3_avg': 2.5,  # Constructor performing well
    'track_history_avg': 2.0,  # Historically good at this track
    'quali_race_ratio': 1.2,  # Typically improves in race
    'circuit_win_rate': 0.3,  # Won 30% of races at this circuit
    'constructor_circuit_avg': 2.8  # Constructor typically performs well here
}

prediction = predict_race_finish(model, scenario)

midfield_scenario = {
    'grid': 12,  # Starting from P12
    'position_quali': 12,
    'last_3_avg_position': 10.0,
    'constructor_last_3_avg': 9.5,
    'track_history_avg': 11.0,
    'quali_race_ratio': 0.9,
    'circuit_win_rate': 0.0,
    'constructor_circuit_avg': 9.5
}

print("\nMidfield Scenario:")
prediction_midfield = predict_race_finish(model, midfield_scenario)
